In [4]:
import pandas as pd
import numpy as np
from numpy import random
import random

### GA Parameter

In [5]:
n_population = 10
interation = 100
n_elitism = 2

### INITIALIZATION


#### Parameter

In [21]:
# number of part
n = 3
# number of assembly tasks
a = 3
m = 5
#number of operations for each part
Q = [3,2,3]
total_ops = sum(Q)
# max of operations
MaxOps = np.max(Q)
# range for operations
O = np.arange(1, MaxOps + 1)
# Processing time of operation 𝑗 of part 𝑝 on machine 𝑘
# Tp = np.random.randint(1, 10, size=(n, MaxOps, m))

In [22]:
machine_seq = [random.randint(1, m) for _ in range(total_ops)]

In [23]:
machine_seq

[5, 4, 3, 2, 1, 4, 4, 2]

In [19]:
assembly_seq = random.sample(range(1, a + 1), a)

In [34]:
# Problem parameters
num_parts = 4
max_ops = 3
max_machines = 5
Qp = [3, 3, 3, 3]  # number of operations per part
num_assemblies = 4

# Machine assignment matrix (valid m       achines for part p and op o), 0 = invalid
machine_matrix = [
    [1, 2, 3],
    [3, 2, 0],
    [2, 0, 1],
    [1, 2, 3]
]
part_seq = random.sample(range(1, n + 1), n)
print("Part sequence:", part_seq)
machine_seq = []
for p in part_seq:
    for o in range(Qp[p-1]):
        valid_machine = machine_matrix[p-1][o]
        machine_seq.append(valid_machine)
        print(valid_machine)

Part sequence: [2, 1, 3]
3
2
0
1
2
3
2
0
1


In [ ]:
import random

# Problem parameters
num_parts = 4
max_ops = 3
max_machines = 5
Qp = [3, 3, 3, 3]  # number of operations per part
num_assemblies = 4

# Machine assignment matrix (valid m       achines for part p and op o), 0 = invalid
machine_matrix = [
    [1, 2, 3],
    [3, 2, 1],
    [2, 3, 1],
    [1, 2, 3]
]

# Assembly dependencies: each assembly needs all parts
depends_on = [
    [1, 1, 1, 1],
    [1, 1, 1, 1],
    [1, 1, 1, 1],
    [1, 1, 1, 1],
]


def generate_feasible_chromosome():
    part_seq = random.sample(range(1, num_parts + 1), num_parts)
    machine_seq = []
    for p in part_seq:
        for o in range(Qp[p-1]):
            valid_machine = machine_matrix[p-1][o]
            machine_seq.append(valid_machine)
    assembly_seq = random.sample(range(1, num_assemblies + 1), num_assemblies)
    return {
        "part_seq": part_seq,
        "machine_seq": machine_seq,
        "assembly_seq": assembly_seq
    }


def decode_chromosome(chromosome):
    """
    Decodes a chromosome into a schedule: maps each part operation to a machine.
    Returns a list of tuples: (part, operation_index, assigned_machine)
    """
    part_seq = chromosome["part_seq"]
    machine_seq = chromosome["machine_seq"]

    schedule = []
    op_counter = {p: 0 for p in part_seq}  # track which op we're on for each part
    machine_index = 0

    for p in part_seq:
        for o in range(1, Qp[p - 1] + 1):
            if machine_index >= len(machine_seq):
                return None  # malformed chromosome
            assigned_machine = machine_seq[machine_index]
            schedule.append((p, o, assigned_machine))
            machine_index += 1

    return schedule


def check_feasibility(schedule):
    """
    Checks if the schedule is feasible given machine_matrix constraints.
    Ensures every (part, op) is assigned a valid machine.
    """
    for (p, o, m) in schedule:
        valid_machine = machine_matrix[p - 1][o - 1]
        if m != valid_machine:
            return False
    return True


feasible = False

while not feasible:
    chrom = generate_feasible_chromosome()
    decoded = decode_chromosome(chrom)
    # if decoded is None:
    #     continue  # skip malformed chromosomes
    feasible = check_feasibility(decoded)
    if feasible:
        break

# After loop ends, 'chrom' is feasible
print("Feasible chromosome found:")
print("Part sequence:", chrom["part_seq"])
print("Machine sequence:", chrom["machine_seq"])
print("Assembly sequence:", chrom["assembly_seq"])
print("Decoded:", decoded)

Feasible chromosome found:
Part sequence: [2, 1, 3, 4]
Machine sequence: [3, 2, 1, 1, 2, 3, 2, 3, 1, 1, 2, 3]
Assembly sequence: [2, 1, 4, 3]
Decoded: [(2, 1, 3), (2, 2, 2), (2, 3, 1), (1, 1, 1), (1, 2, 2), (1, 3, 3), (3, 1, 2), (3, 2, 3), (3, 3, 1), (4, 1, 1), (4, 2, 2), (4, 3, 3)]


In [ ]:
import random
import copy

Qp = [3, 2, 3]  # Number of operations per part
n_parts = len(Qp)
MaxOps = max(Qp)
a = 1  # One assembly
dependsOn = [[1, 1, 1]]
asmTime = [5.0]

machine = [
    [1, 2, 3],
    [3, 2, 0],
    [3, 1, 2]
]

procTime = [
    [3.0, 2.0, 2.0],
    [2.0, 4.0, 0.0],
    [4.0, 3.0, 2.0]
]

def generate_random_chromosome(Qp):
    operations = {p+1: [(p+1, o+1) for o in range(Qp[p])] for p in range(len(Qp))}
    ready_ops = [ops.pop(0) for ops in operations.values()]
    chromosome = []

    while ready_ops:
        op = random.choice(ready_ops)
        chromosome.append(op)
        part = op[0]
        if operations[part]:
            ready_ops.append(operations[part].pop(0))
        ready_ops.remove(op)

    return chromosome

def generate_initial_population(pop_size, Qp):
    return [generate_random_chromosome(Qp) for _ in range(pop_size)]

# --- Decoder: Builds a schedule and computes makespan ---
def decode_schedule(chromosome):
    machine_avail = {}  # machine_id -> time
    part_next_op = {p+1: 1 for p in range(len(Qp))}
    op_start = {}       # (p, o) -> start time
    op_finish = {}      # (p, o) -> finish time

    part_finish = {}    # part -> finish time

    for (p, o) in chromosome:
        if o != part_next_op[p]:
            continue  # skip if out-of-order (shouldn’t happen)
        part_next_op[p] += 1

        m = machine[p-1][o-1]
        pt = procTime[p-1][o-1]
        if m == 0:  # No operation
            op_start[(p, o)] = 0.0
            op_finish[(p, o)] = 0.0
            continue

        # Precedence constraint
        prev_finish = op_finish.get((p, o-1), 0.0)

        # Machine constraint
        avail = machine_avail.get(m, 0.0)

        start = max(prev_finish, avail)
        finish = start + pt

        op_start[(p, o)] = start
        op_finish[(p, o)] = finish

        machine_avail[m] = finish

    # Compute part finish time
    for p in range(1, n_parts + 1):
        last_op = Qp[p-1]
        part_finish[p] = op_finish.get((p, last_op), 0.0)

    # Assembly: start after all required parts are ready
    asm_start = max(part_finish[p+1] for p in range(n_parts) if dependsOn[0][p] == 1)
    asm_finish = asm_start + asmTime[0]

    # Inventory waiting time
    inventory_time = sum(asm_start - part_finish[p+1] for p in range(n_parts) if dependsOn[0][p] == 1)

    makespan = asm_finish
    return makespan, inventory_time, asm_start

# --- Genetic Algorithm Components ---

def fitness(chrom):
    makespan, _, _ = decode_schedule(chrom)
    return makespan  # Lower is better

def tournament_selection(pop, k=3):
    selected = random.sample(pop, k)
    return min(selected, key=fitness)

def crossover(parent1, parent2):
    size = len(parent1)
    cut1, cut2 = sorted(random.sample(range(size), 2))
    
    middle = parent1[cut1:cut2]
    rest = [op for op in parent2 if op not in middle]

    child = rest[:cut1] + middle + rest[cut1:]
    return child

def mutate(chrom, mutation_rate=0.2):
    chrom = chrom[:]
    for _ in range(2):
        if random.random() < mutation_rate:
            i, j = sorted(random.sample(range(len(chrom)), 2))
            # Only swap if they are from different parts or non-conflicting
            if chrom[i][0] == chrom[j][0] and chrom[i][1] > chrom[j][1]:
                continue
            chrom[i], chrom[j] = chrom[j], chrom[i]
    return chrom

# --- GA Loop ---

def genetic_algorithm(Qp, pop_size=30, generations=100):
    population = generate_initial_population(pop_size, Qp)
    best = min(population, key=fitness)

    for gen in range(generations):
        new_pop = []
        while len(new_pop) < pop_size:
            p1 = tournament_selection(population)
            p2 = tournament_selection(population)
            child = crossover(p1, p2)
            child = mutate(child)
            new_pop.append(child)
        population = new_pop
        current_best = min(population, key=fitness)
        if fitness(current_best) < fitness(best):
            best = current_best
        if gen % 10 == 0:
            print(f"Generation {gen}: Best makespan = {fitness(best):.2f}")

    return best, decode_schedule(best)

# --- Run the GA ---

if __name__ == "__main__":
    best_sol, (ms, inv, sa) = genetic_algorithm(Qp, pop_size=50, generations=100)
    print("\nBest solution:")
    print(best_sol)
    print(f"Makespan: {ms:.2f}, Inventory Time: {inv:.2f}, Assembly Start: {sa:.2f}")


Generation 0: Best makespan = 5.00
Generation 10: Best makespan = 5.00
Generation 20: Best makespan = 5.00
Generation 30: Best makespan = 5.00
Generation 40: Best makespan = 5.00
Generation 50: Best makespan = 5.00
Generation 60: Best makespan = 5.00
Generation 70: Best makespan = 5.00
Generation 80: Best makespan = 5.00
Generation 90: Best makespan = 5.00

Best solution:
[(1, 3), (3, 2), (1, 1), (3, 1), (2, 2), (1, 2), (3, 3), (2, 1)]
Makespan: 5.00, Inventory Time: 0.00, Assembly Start: 0.00


### Variable

##### Time Variable JSSP

In [73]:
# time variable
starting_time_of_operation = np.zeros((n, MaxOps, m))
completion_time_of_operation = np.zeros((n, MaxOps, m))
completion_time_of_processing = np.zeros((n, MaxOps, m))

#### Assembly Variable 

In [74]:
# assembly variable
# part_in_position = np.zeros()
starting_time_of_part = np.zeros((n, m))
completion_time_of_part =   np.zeros((n, m))
inventory_time_of_part = np.zeros((n, m))

## SOLUTION PRESENTATION


### Machine INIT


In [75]:
machine_init = np.random.permutation(np.arange(1, ((MaxOps*m)+1)))

### Part INIT


In [76]:
part_init = np.random.permutation(np.arange(1, n+1))

### Assembly INIT


In [77]:
assembly_init = np.random.permutation(np.arange(1, m+1))

### Full Array 

In [78]:
init = np.concatenate((machine_init, part_init, assembly_init), axis=0)

### Create pop

In [79]:
pop = []

for i in range(1, n_population + 1):
    machine_init = np.random.permutation(np.arange(1, (MaxOps * m) + 1))
    part_init = np.random.permutation(np.arange(1, n + 1))
    assembly_init = np.random.permutation(np.arange(1, n + 1))
    init = np.concatenate((machine_init, part_init, assembly_init), axis=0)
    
    pop.append(init)


In [80]:
pop

[array([9, 6, 1, 2, 5, 7, 4, 8, 3, 3, 2, 1, 4, 4, 3, 1, 2]),
 array([2, 6, 3, 5, 1, 9, 4, 7, 8, 2, 4, 1, 3, 4, 1, 3, 2]),
 array([6, 8, 1, 5, 2, 3, 7, 9, 4, 1, 4, 2, 3, 2, 4, 1, 3]),
 array([4, 8, 2, 9, 7, 3, 5, 6, 1, 2, 3, 1, 4, 2, 4, 3, 1]),
 array([5, 6, 1, 2, 8, 7, 9, 4, 3, 4, 2, 3, 1, 3, 2, 4, 1]),
 array([5, 2, 9, 7, 6, 4, 3, 1, 8, 2, 1, 4, 3, 1, 3, 4, 2]),
 array([8, 4, 5, 9, 2, 7, 6, 3, 1, 3, 2, 4, 1, 1, 2, 3, 4]),
 array([6, 9, 2, 7, 1, 3, 4, 5, 8, 1, 3, 2, 4, 3, 4, 1, 2]),
 array([2, 3, 5, 6, 4, 7, 1, 8, 9, 4, 2, 1, 3, 1, 2, 3, 4]),
 array([2, 6, 3, 8, 9, 4, 5, 1, 7, 1, 3, 4, 2, 3, 2, 1, 4])]

### Decode to calcuation

In [84]:
def decode_schedule(individual):
    machine_schedule = individual[:MaxOps * m].reshape((m, MaxOps))
    part_schedule = individual[MaxOps * m:MaxOps * m + n]
    assembly_schedule = individual[MaxOps * m + n:]
    
    return machine_schedule, part_schedule, assembly_schedule
for individual in pop:
    machine_schedule, part_schedule, assembly_schedule = decode_schedule(individual)
    
    # print("Machine Schedule:\n", machine_schedule)
    # print("Part Schedule:\n", part_schedule)
    # print("Assembly Schedule:\n", assembly_schedule)
print(machine_schedule)
print(part_schedule)
print(assembly_schedule)

[[2 6 3]
 [8 9 4]
 [5 1 7]]
[1 3 4 2]
[3 2 1 4]


### Fitness function

In [54]:
def decode_schedule(individual, PT, Q, M):
    n = len(Q)
    MaxOps = max(Q)
    
    # Start and end times of operations (mocked for now)
    Sp = np.random.randint(0, 20, size=(n, MaxOps))  # Start times
    Ep = Sp + np.random.randint(1, 10, size=(n, MaxOps))  # End times

    # Assembly start and end times
    Sa = np.max(Ep, axis=1) + np.random.randint(1, 5, size=n)
    Ea = Sa + np.random.randint(5, 10, size=n)

    return Sp, Ep, Sa, Ea

def fitness_function(individual, PT, Q, Ta, nd, td, no, to, nt, tt, M=1000):
    n = len(Q)       # number of parts
    MaxOps = max(Q)
    m = PT.shape[2]  # number of machines
    
    # Decode individual into scheduling
    # You need to write: decode_schedule(individual) → gives Sp, Ep, Sa, Ea, etc.
    Sp, Ep, Sa, Ea = decode_schedule(individual, PT, Q, M)
    
    E_part = np.max(Ep, axis=1)  # shape (n,)
    inventory_time = Sa - E_part
    total_inventory_time = np.sum(inventory_time)
    
    penalty_time = nd * td + no * to + nt * tt
    total_penalty_time = np.sum(penalty_time)
    
    makespan = np.max(Ea)
    
    alpha = 1.0  # weight for inventory
    beta = 1.0   # weight for penalties
    
    fitness = makespan + alpha * total_inventory_time + beta * total_penalty_time
    return fitness


In [83]:
Tp

array([[[1, 2, 8],
        [4, 8, 7],
        [6, 5, 6]],

       [[1, 3, 5],
        [7, 1, 2],
        [3, 7, 4]],

       [[5, 3, 8],
        [3, 5, 7],
        [3, 3, 2]],

       [[3, 4, 2],
        [6, 7, 2],
        [7, 8, 4]]])

In [55]:
fitness = fitness_function(individual, PT, Q, Ta, nd, td, no, to, nt, tt)
print(f"Fitness value: {fitness}")


Fitness value: 56.0
